# Wrapper

Use case: When collecting show commands from a switch for debugging, it would be nice to have a timestamp and to include the actual command besides the output. In addition, to delineate each command sections so that they are easy to find when parsing the results, let's prepend each output with a comment string such as ### <my command>


1. Read content of file containing show commands "cli.txt"

```python
        with open(file="cli.txt", mode="r", encoding="UTF-8") as f_read:
            first_line = str(f_reader.readline().strip())
```

2. For each nxos <my show command> line generate the following section

```python
        echo "### <my show command>" >> $(SWITCHNAME)-logs.txt
        show clock >> $(SWITCHNAME)-logs.txt
        <my show command>  >> $(SWITCHNAME)-logs.txt
```

3. improvement - redirect those outputs to a file (in addition to printing to screen): wrapped_cli.txt
  * first line should be an overwrite redirect
  * next lines should append to the file

```python
        my_stuff = '''
        this is an example
        '''
        with open(file="output.txt", mode="w", encoding="UTF-8") as f_write:
            f_write.write(my_stuff)
```

### 1. Wrap the initial string command

In [3]:
# Given a string, generate a new block with comment <string> , show clock and <string> 
command = "show version"

wrapped_command = "### %s" % command
wrapped_command += '\n'  + "show clock\n"
wrapped_command += command

In [4]:
print(wrapped_command)

### show version
show clock
show version


### 2. Turn it into a function

since we plan to re-use this block... we can create a function

In [5]:
def wrapper(command: str) -> str:
    # first line is a comment showing the command name
    wrapped_command = "### %s" % command + '\n'
    wrapped_command += "show clock" + '\n'
    wrapped_command += command
    return wrapped_command

In [6]:
print(wrapper("show module"))

### show module
show clock
show module


### 3. Read a list of commands from file

use ```cli.txt``` for the files were commands are located.

In [11]:
show_commands_file = 'cli.txt'

# Read the commands
with open(file=show_commands_file, mode='r', encoding='UTF-8') as f_read:
    # readlines automatically split the lines into a list
    show_commands = f_read.readlines()

# Check the commands
print(show_commands)

# remove the end of line character
show_commands = [ cli.strip() for cli in show_commands]

# Check the commands again
print(show_commands)


['show version\n', 'show module\n', 'show logg log']
['show version', 'show module', 'show logg log']


### 4. Wrap the commands

In [12]:
# need to remove the end of line character '\n' from the c
wrapped_commands = [ wrapper(cli) for cli in show_commands ]
print(wrapped_commands)

['### show version\nshow clock\nshow version', '### show module\nshow clock\nshow module', '### show logg log\nshow clock\nshow logg log']


### 5. Write strings to file

In [17]:
# to assign multiple lines of text to a variable, use the ''' '''
example = '''LEAF47# sh module
Mod Ports             Module-Type                      Model            Status
--- ----- --------------------------------------- --------------------- --------
1    54   48x10/25G + 6x40/100G Ethernet Module   N9K-C93180YC-EX       active *

Mod  Sw                       Hw    Slot
---  ----------------------- ------ ----
1    10.3(2)                  3.0    NA


Mod  MAC-Address(es)                         Serial-Num
---  --------------------------------------  ----------
1    00-6b-f1-84-02-90 to 00-6b-f1-84-02-df  FDO20380BNT

Mod  Online Diag Status
---  ------------------
1    Pass

* this terminal session
'''

output_file = 'show_module.txt'

# write this output to file
with open(file=output_file, mode='w', encoding='UTF-8') as f_write:
    f_write.write(example)

# Check file was created
import os
os.path.isfile(output_file)


True

In [25]:
# Verify output with readlines (split the output into a list of individual lines including '\n')
with open(file=output_file, mode='r', encoding='UTF-8') as f_read:
    # readlines automatically split the lines into a list
    multiple_lines = f_read.readlines()
    one_section = f_read.readline()

print("multiple lines:")
print(multiple_lines)


multiple lines:
['LEAF47# sh module\n', 'Mod Ports             Module-Type                      Model            Status\n', '--- ----- --------------------------------------- --------------------- --------\n', '1    54   48x10/25G + 6x40/100G Ethernet Module   N9K-C93180YC-EX       active *\n', '\n', 'Mod  Sw                       Hw    Slot\n', '---  ----------------------- ------ ----\n', '1    10.3(2)                  3.0    NA\n', '\n', '\n', 'Mod  MAC-Address(es)                         Serial-Num\n', '---  --------------------------------------  ----------\n', '1    00-6b-f1-84-02-90 to 00-6b-f1-84-02-df  FDO20380BNT\n', '\n', 'Mod  Online Diag Status\n', '---  ------------------\n', '1    Pass\n', '\n', '* this terminal session\n']


In [24]:
with open(file=output_file, mode='r', encoding='UTF-8') as f_read:
    first_line = f_read.readline()
    print(first_line)

LEAF47# sh module

